In [1]:
import os
import optuna
from dotenv import load_dotenv
load_dotenv()

/home/jsimonrichard/QML_Research/QFE-Experiments/qfe_exp_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
optuna.get_all_study_names(storage=os.getenv('OPTUNA_DB'))

['PROTEINS_none_simple-pooling-0.2_test',
 'PROTEINS_MLP-D_simple-pooling-0.2_test',
 'PROTEINS_MLP-2^D_simple-pooling-0.2_test',
 'PROTEINS_QFE-exp_simple-pooling-0.1_test',
 'PROTEINS_QFE-probs_simple-pooling-0.1_test',
 'PROTEINS_MLP-2^D_simple-pooling-0.1_test',
 'PROTEINS_none_simple-pooling_0.1-test',
 'PROTEINS_QFE-exp_simple-pooling-0.2_test',
 'PROTEINS_QFE-probs_simple-pooling-0.2_test']

In [4]:
study = optuna.load_study(study_name="PROTEINS_none_simple-pooling-0.2_test", storage=os.getenv('OPTUNA_DB'))
study.best_trials

[FrozenTrial(number=22, state=TrialState.COMPLETE, values=[0.6420929858429859, 0.01033622908622911, 14742.0], datetime_start=datetime.datetime(2024, 5, 4, 13, 52, 39, 424360), datetime_complete=datetime.datetime(2024, 5, 4, 13, 54, 6, 7860), params={'model': 'GCN', 'pooling': 'max', 'layers': 2, 'hidden': 91, 'dropout': 0.2241991518928222, 'lr': 0.0003625937432164209, 'weight_decay': 1.9334604700063187e-05, 'batch_size': 1024}, user_attrs={}, system_attrs={'nsga2:generation': 0}, intermediate_values={}, distributions={'model': CategoricalDistribution(choices=('GCN', 'GraphConv', 'GraphSAGE', 'GAT')), 'pooling': CategoricalDistribution(choices=('sum', 'mean', 'max')), 'layers': IntDistribution(high=16, log=False, low=1, step=1), 'hidden': IntDistribution(high=256, log=False, low=16, step=1), 'dropout': FloatDistribution(high=0.5, log=False, low=0.0, step=None), 'lr': FloatDistribution(high=0.01, log=True, low=1e-05, step=None), 'weight_decay': FloatDistribution(high=0.001, log=True, low

In [5]:
from optuna.visualization import plot_pareto_front

In [6]:
plot_pareto_front(study)

In [8]:
import numpy as np

In [19]:
accuracies = np.array([trial.values[0] for trial in study.best_trials])
errors = np.array([trial.values[1] for trial in study.best_trials])
f1s = np.array([trial.values[2] for trial in study.best_trials])

accuracies_mean = accuracies.mean()
accuracies_std = accuracies.std()
errors_mean = errors.mean()
errors_std = errors.std()
f1s_mean = f1s.mean()
fls_std = f1s.std()

def utility(trial):
    z_acc = (trial.values[0] - accuracies_mean) / accuracies_std
    z_err = -(trial.values[1] - errors_mean) / errors_std
    z_f1 = -(trial.values[2] - f1s_mean) / fls_std

    return z_acc + 3*z_f1 #+ z_err + z_f1

best_trial = max(study.best_trials, key=utility)
best_trial.values

[0.7292939293162903, 0.026028163550524674, 38880.0]

In [20]:
best_trial.number

221